# Running Marlowe with a Reference Script

**Executive Summary**

This example shows how to use the `marlowe-cli` and `cardano-cli` tools to submit Marlowe transactions that involve reference scripts. The `marlowe-cli` tool is used to generate the script, datum, and redeemers, while the `cardano-cli` tool is used to submit transactions and query the blockchain. Using a reference script significantly reduces the cost of a Marlowe transaction.

## Preliminaries

Record the versions of the tools used.

In [1]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [2]:
marlowe-cli --version

marlowe-cli 0.0.7.0 @ a2ad214dd7930d9f57db768cdf1852ff938cf460


## 1. Select the network

We use the `preview` network. See [the configuration files](https://book.world.dev.cardano.org/environments.html#preview-testnet) and use [faucet](https://faucet.preview.world.dev.cardano.org/basic-faucet).

In [3]:
export CARDANO_TESTNET_MAGIC=2

In [4]:
export CARDANO_NODE_SOCKET_PATH=node.socket

Find the current tip

In [5]:
TIP=$(cardano-cli query tip --testnet-magic "$CARDANO_TESTNET_MAGIC" | jq .slot)
echo "The tip of the blockchain is at slot $TIP."

The tip of the blockchain is at slot 3007240.


Retrieve the protocol parameters.

In [6]:
cardano-cli query protocol-parameters --testnet-magic "$CARDANO_TESTNET_MAGIC" \
                                      --out-file reference-script.protocol

We've empirically determined the slotting parameters for this network.

In [7]:
SLOT_OFFSET=$((1660003200 * 1000))  # POSIX milliseconds for slot zero
SLOT_SPACING=1000                   # milliseconds per slot

In [8]:
echo $SLOT_OFFSET

1660003200000


Convert a slot number to POSIX milliseconds.

In [9]:
function slot2time () {
  echo $(($1 * SLOT_SPACING + SLOT_OFFSET))
}

Convert POSIX milliseconds to a slot number.

In [10]:
function time2slot () {
  echo $((($1 - SLOT_OFFSET) / SLOT_SPACING))
}

Test the time conversion functions.

In [11]:
NOW=$(slot2time $TIP)
echo $NOW

1663010440000


In [12]:
echo "Compare $(time2slot $NOW) to $TIP."

Compare 3007240 to 3007240.


Define some time constants, for convenience.

In [13]:
SECOND=1000              # milliseconds per second
MINUTE=$((60 * SECOND))  # milliseconds per minute
HOUR=$((60 * MINUTE))    # milliseconds per hour
DAY=$((24 * HOUR))       # milliseconds per day

Define a constant for Ada.

In [14]:
ADA=1000000  # Lovelace per Ada

## 2. Select the signing and payment keys

Use pre-existing stakeless keys for this example.

In [15]:
PAYMENT_SKEY=payment.skey
PAYMENT_VKEY=payment.vkey

Compute the address.

In [16]:
ADDRESS_P=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$PAYMENT_VKEY")
echo $ADDRESS_P

addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


Compute the public key hash.

In [17]:
PUBKEYHASH_P=$(cardano-cli address key-hash --payment-verification-key-file "$PAYMENT_VKEY")
echo $PUBKEYHASH_P

0a11b0c7e25dc5d9c63171bdf39d9741b901dc903e12b4e162348e07


Find a UTxO that we can use the fund the contract.

In [18]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADDRESS_P"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
97311e871a028f2bc726c1306aea6fa5c866303a508c6282e5e07fb8223a3922     0        179425065440 lovelace + TxOutDatumNone


Select a UTxO to fund the later transactions.

In [19]:
TX_0="97311e871a028f2bc726c1306aea6fa5c866303a508c6282e5e07fb8223a3922#0"

## 3. Design the Marlowe contract

For this demonstraction we just use a simple contract that involves two notifications and the refund of the Ada at the script address. Use the [Marlowe Playground](https://marlowe-playground-staging.plutus.aws.iohkdev.io/#/blockly) to create this contract and the "Download JSON" button to access the JSON representation of the contract.

![Blockly for the two-notify contract](reference-script.png)

We customize the downloaded JSON with our own timeout values.

In [20]:
TIMEOUT=$((NOW + 30 * DAY))
echo $TIMEOUT

1665602440000


### 3.1. Creation

The initial contract has two notifications.

In [21]:
cat > reference-script-1.contract << EOI
{
  "when" : [
    {
      "case" : { "notify_if" : true },
      "then" : {
        "when" : [
          {
            "case" : { "notify_if" : true },
            "then" : "close"
          }
        ],
        "timeout": $TIMEOUT,
        "timeout_continuation" : "close"
      }
    }
  ],
  "timeout" : $TIMEOUT,
  "timeout_continuation" : "close"
}
EOI
json2yaml reference-script-1.contract

timeout: 1665602440000
timeout_continuation: close
when:
- case:
    notify_if: true
  then:
    timeout: 1665602440000
    timeout_continuation: close
    when:
    - case:
        notify_if: true
      then: close


We set the initial state to hold a small amount of Ada.

In [22]:
ACCOUNT_LOVELACE=$((3 * ADA))

In [23]:
cat << EOI > reference-script-1.state
{
  "accounts" : [
    [
      [
        { "pk_hash" : "$PUBKEYHASH_P" },
        { "currency_symbol" : "", "token_name" : "" }
      ],
      $ACCOUNT_LOVELACE
    ]
  ],
  "choices" : [],
  "boundValues" : [],
  "minTime" : $NOW
}
EOI
json2yaml reference-script-1.state

accounts:
- - - pk_hash: 0a11b0c7e25dc5d9c63171bdf39d9741b901dc903e12b4e162348e07
    - currency_symbol: ''
      token_name: ''
  - 3000000
boundValues: []
choices: []
minTime: 1663010440000


### 3.2. First notification

We send an `INotify` to the contract.

In [24]:
marlowe-cli input notify > reference-script-1.input
json2yaml reference-script-1.input

input_notify
...


The contract then becomes much simpler.

In [25]:
cat > reference-script-2.contract << EOI
{
  "when" : [
    {
      "case" : { "notify_if" : true },
      "then" : "close"
    }
  ],
  "timeout" : $TIMEOUT,
  "timeout_continuation" : "close"
}
EOI
json2yaml reference-script-2.contract

timeout: 1665602440000
timeout_continuation: close
when:
- case:
    notify_if: true
  then: close


We plan to submit the input within the following validity range.

In [26]:
INVALID_BEFORE=$(time2slot $NOW)
INVALID_HEREAFTER=$(time2slot $((NOW + 5 * HOUR)))
echo "$INVALID_BEFORE $INVALID_HEREAFTER"

3007240 3025240


This means that the state after the first input will be the same as the previous state, since no adjustment of `minTime` occurs due to this validity range.

In [27]:
cat << EOI > reference-script-2.state
{
  "accounts" : [
    [
      [
        { "pk_hash" : "$PUBKEYHASH_P" },
        { "currency_symbol" : "", "token_name" : "" }
      ],
      $ACCOUNT_LOVELACE
    ]
  ],
  "choices" : [],
  "boundValues" : [],
  "minTime" : $NOW
}
EOI
json2yaml reference-script-2.state

accounts:
- - - pk_hash: 0a11b0c7e25dc5d9c63171bdf39d9741b901dc903e12b4e162348e07
    - currency_symbol: ''
      token_name: ''
  - 3000000
boundValues: []
choices: []
minTime: 1663010440000


### 3.3. Second notification and closure

We send another `INotify` to the contract, which causes it to close.

In [28]:
marlowe-cli input notify > reference-script-2.input
json2yaml reference-script-2.input

input_notify
...


The contract just becomes `Close`.

In [29]:
cat > reference-script-3.contract << EOI
"close"
EOI
json2yaml reference-script-3.contract

close
...


The state becomes empty.

In [30]:
cat << EOI > reference-script-3.state
{
  "accounts" : [],
  "choices" : [],
  "boundValues" : [],
  "minTime" : $NOW
}
EOI
json2yaml reference-script-3.state

accounts: []
boundValues: []
choices: []
minTime: 1663010440000


## 4. Run the transactions

### 4.1 Store the reference script on chain

Compute the address of the Marlowe validator.

In [31]:
ADDRESS_S=$(marlowe-cli contract address)
echo $ADDRESS_S

addr_test1wzvmq4tgspn7sdps4dhqf02ta28jetz8drcwpxlaytp0z2ga35plq


Export the Marlowe validator itself.

In [32]:
marlowe-cli contract validator --out-file reference-script.plutus

addr_test1wzvmq4tgspn7sdps4dhqf02ta28jetz8drcwpxlaytp0z2ga35plq


In [33]:
head -c 1000 reference-script.plutus

{
    "type": "PlutusScriptV2",
    "description": "",
    "cborHex": "59316259315f01000033232323322323232323232323232323232323233223232323232323232323232323322323232323232323232332233223232323232323322323232323232323232323232332233223232323232323232323232323232323232323232323232323232323232323232323232323322323232323232323232323232323232323232323223222232325335333222350032232322350062323232323223353235001223500223223355335333573466e2000400c22c04228044c0d0c8488c00400ccd542180400c00454cd4ccd5cd19b88001501008b0108a0113034332212233002004003501033550860100300113322122330010040033355086015002001350112222333308901004003002500623033122222300200622533335333333305308c01002001010006509d01509d01130341222220051303412222200313034122222004222221533500513333038004003002001153333335015221303b03c13501822225335333355307612001505e2209701004096011303d03e1333303c0080070060052221303c03d2221303c03d222221303e03f2221303c03d15335333573466e2400540382240422004540384004cc8848cc00400c008d4d401c88888888888801488d400

Build the transaction to store the reference script for Marlowe on the blockchain.

In [34]:
cardano-cli transaction build --testnet-magic "$CARDANO_TESTNET_MAGIC"                 \
                              --babbage-era                                            \
                              --protocol-params-file reference-script.protocol         \
                              --tx-in "$TX_0"                                          \
                              --tx-out "$ADDRESS_P+$((60 * ADA))"                      \
                                --tx-out-reference-script-file reference-script.plutus \
                              --change-address "$ADDRESS_P"                            \
                              --out-file tx-1.raw

Estimated transaction fee: Lovelace 722453


Sign the transaction.

In [35]:
cardano-cli transaction sign --testnet-magic "$CARDANO_TESTNET_MAGIC" \
                             --tx-body-file tx-1.raw                  \
                             --signing-key-file "$PAYMENT_SKEY"       \
                             --out-file tx-1.signed

Submit the transaction, and wait for it to be confirmed.

In [36]:
cardano-cli transaction submit --testnet-magic "$CARDANO_TESTNET_MAGIC" \
                               --tx-file tx-1.signed

Transaction successfully submitted.


In [37]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADDRESS_P"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
2fe958006846ee4e355835b6a367b0591752ad5d9b32b854e0c670e42066a1e5     0        179364342987 lovelace + TxOutDatumNone
2fe958006846ee4e355835b6a367b0591752ad5d9b32b854e0c670e42066a1e5     1        60000000 lovelace + TxOutDatumNone


Record the transaction ID and the ID of the UTxO holding the reference script.

In [38]:
TX_1=2fe958006846ee4e355835b6a367b0591752ad5d9b32b854e0c670e42066a1e5
TX_S="$TX_1#1"

### 4.2. Creation

Bundle the contract and state, computing the datum and script.

In [39]:
marlowe-cli contract marlowe --testnet-magic "$CARDANO_TESTNET_MAGIC"    \
                             --contract-file reference-script-1.contract \
                             --state-file    reference-script-1.state    \
                             --out-file      reference-script-1.marlowe  \
                             --print-stats


Bare-validator cost: ExBudget {exBudgetCPU = ExCPU 18515100, exBudgetMemory = ExMemory 80600}
Validator size: 12642
Datum size: 139
Redeemer size: 3
Total size: 12784


Extract the datum.

In [40]:
jq '.datum.json' reference-script-1.marlowe > reference-script-1.datum

Build the transaction to create the Marlowe contract.

In [41]:
cardano-cli transaction build --testnet-magic "$CARDANO_TESTNET_MAGIC"             \
                              --babbage-era                                        \
                              --protocol-params-file reference-script.protocol     \
                              --tx-in "$TX_1#0"                                    \
                              --tx-out "$ADDRESS_S+$ACCOUNT_LOVELACE"              \
                                --tx-out-datum-embed-file reference-script-1.datum \
                              --change-address "$ADDRESS_P"                        \
                              --out-file tx-2.raw

Estimated transaction fee: Lovelace 175005


Sign the transaction.

In [42]:
cardano-cli transaction sign --testnet-magic "$CARDANO_TESTNET_MAGIC" \
                             --tx-body-file tx-2.raw                  \
                             --signing-key-file "$PAYMENT_SKEY"       \
                             --out-file tx-2.signed

Submit the transaction.

In [43]:
cardano-cli transaction submit --testnet-magic "$CARDANO_TESTNET_MAGIC" \
                               --tx-file tx-2.signed

Transaction successfully submitted.


Wait for the transaction to be confirmed. (One can check the node logs to see when the confirmation has occurred.)

See that the change has been received

In [44]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADDRESS_P"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
2fe958006846ee4e355835b6a367b0591752ad5d9b32b854e0c670e42066a1e5     1        60000000 lovelace + TxOutDatumNone
e1c9a68b5117c1a9b6510ebba8402c32f6049f893f6b73917155df77ca90b231     0        179361167982 lovelace + TxOutDatumNone


Record the transaction ID.

In [45]:
TX_2=e1c9a68b5117c1a9b6510ebba8402c32f6049f893f6b73917155df77ca90b231

View the transaction at the script address.

In [46]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADDRESS_S" | sed -n -e "1,2p;/^$TX_2/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
e1c9a68b5117c1a9b6510ebba8402c32f6049f893f6b73917155df77ca90b231     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "57789c29413adcaa4132e233245bf076e8a03d190cf1ea89ca0041c7cf037b57"


### 4.3. First notification

Bundle the contract and state, computing the redeemer, datum, and script.

In [47]:
marlowe-cli contract marlowe --testnet-magic "$CARDANO_TESTNET_MAGIC"    \
                             --input-file    reference-script-1.input    \
                             --contract-file reference-script-2.contract \
                             --state-file    reference-script-2.state    \
                             --out-file      reference-script-2.marlowe  \
                             --print-stats


Bare-validator cost: ExBudget {exBudgetCPU = ExCPU 18515100, exBudgetMemory = ExMemory 80600}
Validator size: 12642
Datum size: 109
Redeemer size: 11
Total size: 12762


Extract the redeemer.

In [48]:
jq '.redeemer.json' reference-script-2.marlowe > reference-script-1.redeemer

Extract the datum.

In [49]:
jq '.datum.json' reference-script-2.marlowe > reference-script-2.datum

Build the transaction

In [50]:
cardano-cli transaction build --testnet-magic "$CARDANO_TESTNET_MAGIC"                                \
                              --babbage-era                                                           \
                              --protocol-params-file reference-script.protocol                        \
                              --tx-in-collateral "$TX_2#0"                                            \
                              --tx-in "$TX_2#0"                                                       \
                              --tx-in "$TX_2#1"                                                       \
                                --spending-tx-in-reference "$TX_S"                                    \
                                --spending-plutus-script-v2                                           \
                                --spending-reference-tx-in-datum-file    reference-script-1.datum     \
                                --spending-reference-tx-in-redeemer-file reference-script-1.redeemer  \
                              --tx-out "$ADDRESS_S+$ACCOUNT_LOVELACE"                                 \
                                --tx-out-datum-embed-file reference-script-2.datum                    \
                              --change-address "$ADDRESS_P"                                           \
                              --required-signer "$PAYMENT_SKEY"                                       \
                              --invalid-before "$INVALID_BEFORE"                                      \
                              --invalid-hereafter "$INVALID_HEREAFTER"                                \
                              --out-file tx-3.raw

Estimated transaction fee: Lovelace 432682


Sign and submit the transaction.

In [51]:
cardano-cli transaction sign --testnet-magic "$CARDANO_TESTNET_MAGIC" \
                             --tx-body-file tx-3.raw                  \
                             --signing-key-file "$PAYMENT_SKEY"       \
                             --out-file tx-3.signed

In [52]:
cardano-cli transaction submit --testnet-magic "$CARDANO_TESTNET_MAGIC" \
                               --tx-file tx-3.signed

Transaction successfully submitted.


Wait for the transaction to be confirmed. (One can check the node logs to see when the confirmation has occurred.)

See that the change has been received

In [53]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADDRESS_P"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
2fe958006846ee4e355835b6a367b0591752ad5d9b32b854e0c670e42066a1e5     1        60000000 lovelace + TxOutDatumNone
ab085283a6f0c61447f050d80f06f1e05ea96f40615cd1c574d948ca26bf789e     0        179360735300 lovelace + TxOutDatumNone


Record the transaction ID.

In [54]:
TX_3=ab085283a6f0c61447f050d80f06f1e05ea96f40615cd1c574d948ca26bf789e

View the transaction at the script address.

In [55]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADDRESS_S" | sed -n -e "1,2p;/^$TX_2/p;/^$TX_3/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ab085283a6f0c61447f050d80f06f1e05ea96f40615cd1c574d948ca26bf789e     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "3979826f54ad1ffe5d48ecccd51ab2e63bdb57ef92b1385956b87edad86a0e54"


### 4.4. Second notification

Bundle the contract and state, computing the redeemer, datum, and script.

In [56]:
marlowe-cli contract marlowe --testnet-magic "$CARDANO_TESTNET_MAGIC"    \
                             --input-file    reference-script-2.input    \
                             --contract-file reference-script-3.contract \
                             --state-file    reference-script-3.state    \
                             --out-file      reference-script-3.marlowe  \
                             --print-stats


Bare-validator cost: ExBudget {exBudgetCPU = ExCPU 18515100, exBudgetMemory = ExMemory 80600}
Validator size: 12642
Datum size: 30
Redeemer size: 11
Total size: 12683


Extract the redeemer.

In [57]:
jq '.redeemer.json' reference-script-3.marlowe > reference-script-2.redeemer

No datum is needed because the contract does not continue.

Build the transaction

In [58]:
cardano-cli transaction build --testnet-magic "$CARDANO_TESTNET_MAGIC"                                \
                              --babbage-era                                                           \
                              --protocol-params-file reference-script.protocol                        \
                              --tx-in-collateral "$TX_3#0"                                            \
                              --tx-in "$TX_3#0"                                                       \
                              --tx-in "$TX_3#1"                                                       \
                                --spending-tx-in-reference "$TX_S"                                    \
                                --spending-plutus-script-v2                                           \
                                --spending-reference-tx-in-datum-file    reference-script-2.datum     \
                                --spending-reference-tx-in-redeemer-file reference-script-2.redeemer  \
                              --tx-out "$ADDRESS_P+$ACCOUNT_LOVELACE"                                 \
                              --change-address "$ADDRESS_P"                                           \
                              --required-signer "$PAYMENT_SKEY"                                       \
                              --invalid-before "$INVALID_BEFORE"                                      \
                              --invalid-hereafter "$INVALID_HEREAFTER"                                \
                              --out-file tx-4.raw

Estimated transaction fee: Lovelace 365677


Sign and submit the transaction.

In [59]:
cardano-cli transaction sign --testnet-magic "$CARDANO_TESTNET_MAGIC" \
                             --tx-body-file tx-4.raw                  \
                             --signing-key-file "$PAYMENT_SKEY"       \
                             --out-file tx-4.signed

In [60]:
cardano-cli transaction submit --testnet-magic "$CARDANO_TESTNET_MAGIC" \
                               --tx-file tx-4.signed

Transaction successfully submitted.


Wait for the transaction to be confirmed. (One can check the node logs to see when the confirmation has occurred.)

See that the payment and change have been received

In [61]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADDRESS_P"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
2fe958006846ee4e355835b6a367b0591752ad5d9b32b854e0c670e42066a1e5     1        60000000 lovelace + TxOutDatumNone
3e535eb2de5e8f032c9285df9fff37efba3a3f4a41c0efe235bf656a4d521d40     0        179360369623 lovelace + TxOutDatumNone
3e535eb2de5e8f032c9285df9fff37efba3a3f4a41c0efe235bf656a4d521d40     1        3000000 lovelace + TxOutDatumNone


Record the transaction ID.

In [62]:
TX_4=3e535eb2de5e8f032c9285df9fff37efba3a3f4a41c0efe235bf656a4d521d40

Verify that this contract has no UTxO at the script address.

In [63]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADDRESS_S" | sed -n -e "1,2p;/^$TX_2/p;/^$TX_3/p;/^$TX_4/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


### 4.5 Cleanup

In [64]:
marlowe-cli util clean --required-signer "$PAYMENT_SKEY" \
                       --change-address "$ADDRESS_P"     \
                       --out-file /dev/null              \
                       --submit 600

TxId "91343f2991f1713b4404cd870a8c1599f83c0d2d8fa1a43c5038b678b19c25a8"


In [65]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADDRESS_P"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
91343f2991f1713b4404cd870a8c1599f83c0d2d8fa1a43c5038b678b19c25a8     0        179423191670 lovelace + TxOutDatumNone
